In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "../../data/Landslide2.csv"
device = "101"
save_path = f"../../data/dev{device}_resample10T.csv"
#save_path = f"data/devAll2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo


df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_7840\2983835685.py:12: DtypeWarning: Columns (9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
0,2025-06-27 18:59:34.950000,101.0,41.500000,0.0,28.444725,79.391396,12.150635,8.638015,99.898972
2,2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
4,2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
6,2025-05-02 11:42:28.667259,101.0,28.900000,0.0,34.250019,62.839169,15.995875,8.637998,99.899010
8,2025-05-02 11:42:40.117564,101.0,28.799999,0.0,34.185932,62.661785,19.812029,8.637998,99.899010
...,...,...,...,...,...,...,...,...,...
308512,2025-06-29 20:20:49.190000,101.0,27.900000,0.0,29.483482,80.398491,8.771909,8.638011,99.898972
308515,2025-06-29 20:21:12.230000,101.0,27.900000,0.0,29.528877,80.196312,8.352998,8.638011,99.898972
308536,2025-06-29 20:25:27.980000,101.0,27.900000,0.0,29.101625,81.113754,2.490953,8.638010,99.898972
308539,2025-06-29 20:26:02.150000,101.0,27.900000,0.0,29.050890,81.302589,8.617684,8.638011,99.898972


In [9]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,149607,149607.0,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000
mean,2025-06-04 12:12:35.064885248,101.0,48.239375,0.220004,29.755443,75.709608,11.315496,8.638010,99.898969
min,2025-05-02 11:42:06.635276,101.0,27.299999,0.000000,24.279013,46.977417,0.000000,8.637940,99.166664
25%,2025-05-29 17:45:20.274356992,101.0,44.400002,0.000000,27.405968,70.947433,5.311314,8.638001,99.898964
50%,2025-06-10 20:02:30.664009984,101.0,49.000000,0.000000,28.474098,79.242622,11.374877,8.638014,99.898972
75%,2025-06-17 11:49:32.409999872,101.0,55.400002,0.000000,32.319370,80.932556,15.343357,8.638022,99.898987
max,2025-06-29 20:26:13.090000,101.0,66.599998,41.630600,38.586636,87.270775,32.062466,8.638072,99.899498
std,NaN,0.0,9.134768,1.692000,3.059886,7.981286,6.957680,0.000019,0.001893


In [10]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
#df = df.sort_values(['devID', 'timestamp']).reset_index(drop=True)

# ตั้ง timestamp เป็น index
df = df.set_index('timestamp')
df

,devID,soil,rain,temp,humi,geo,lat,lng
timestamp,,,,,,,,
2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
2025-05-02 11:42:28.667259,101.0,28.900000,0.0,34.250019,62.839169,15.995875,8.637998,99.899010
2025-05-02 11:42:40.117564,101.0,28.799999,0.0,34.185932,62.661785,19.812029,8.637998,99.899010
2025-05-02 11:42:50.732852,101.0,28.900000,0.0,34.175251,63.481953,8.099698,8.637998,99.899010
...,...,...,...,...,...,...,...,...
2025-06-29 20:20:49.190000,101.0,27.900000,0.0,29.483482,80.398491,8.771909,8.638011,99.898972
2025-06-29 20:21:12.230000,101.0,27.900000,0.0,29.528877,80.196312,8.352998,8.638011,99.898972
2025-06-29 20:25:27.980000,101.0,27.900000,0.0,29.101625,81.113754,2.490953,8.638010,99.898972


In [11]:
window = '10T'  # 10 นาที
features = ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']

df_rolling = df[features].resample(window).agg(['mean', 'std', 'min', 'max'])
df_rolling.columns = ['_'.join(col) for col in df_rolling.columns]
df_rolling['devID'] = int(device)
df_rolling = df_rolling.dropna()

df_rolling = df_rolling.reset_index()
df_rolling

C:\Users\ahmad\AppData\Local\Temp\ipykernel_7840\3653440406.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_rolling = df[features].resample(window).agg(['mean', 'std', 'min', 'max'])


,timestamp,soil_mean,soil_std,soil_min,soil_max,rain_mean,rain_std,rain_min,rain_max,temp_mean,...,geo_max,lat_mean,lat_std,lat_min,lat_max,lng_mean,lng_std,lng_min,lng_max,devID
0,2025-05-02 11:40:00,28.807142,0.026066,28.799999,28.900000,0.0,0.0,0.0,0.0,34.483864,...,19.812029,8.637995,1.550673e-06,8.637993,8.637999,99.899005,0.000004,99.899002,99.899010,101
1,2025-05-02 11:50:00,28.796363,0.018892,28.700001,28.799999,0.0,0.0,0.0,0.0,34.705092,...,19.805017,8.637991,2.022327e-06,8.637987,8.637995,99.899003,0.000002,99.899002,99.899010,101
2,2025-05-02 12:00:00,28.757143,0.049934,28.700001,28.799999,0.0,0.0,0.0,0.0,35.428349,...,19.796062,8.637991,2.108593e-06,8.637988,8.637995,99.899009,0.000002,99.899002,99.899010,101
3,2025-05-02 12:10:00,28.680001,0.080277,28.500000,28.799999,0.0,0.0,0.0,0.0,34.673631,...,31.987383,8.637989,3.031666e-06,8.637985,8.637995,99.899014,0.000004,99.899010,99.899017,101
4,2025-05-02 12:20:00,28.487037,0.075352,28.400000,28.700001,0.0,0.0,0.0,0.0,34.498658,...,31.980288,8.637986,2.691552e-06,8.637982,8.637990,99.899014,0.000004,99.899010,99.899017,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180,2025-06-29 19:40:00,27.996000,0.020000,27.900000,28.000000,0.0,0.0,0.0,0.0,29.631097,...,31.980068,8.638011,1.888961e-06,8.638007,8.638014,99.898986,0.000002,99.898979,99.898987,101
3181,2025-06-29 19:50:00,27.991667,0.028233,27.900000,28.000000,0.0,0.0,0.0,0.0,29.709013,...,19.518534,8.638010,4.205846e-06,8.638004,8.638017,99.898985,0.000003,99.898979,99.898987,101
3182,2025-06-29 20:00:00,27.913043,0.034435,27.900000,28.000000,0.0,0.0,0.0,0.0,29.834574,...,31.988052,8.638007,3.455662e-06,8.638004,8.638018,99.898987,0.000000,99.898987,99.898987,101
3183,2025-06-29 20:10:00,27.900000,0.000000,27.900000,27.900000,0.0,0.0,0.0,0.0,29.834822,...,31.985922,8.638013,2.712515e-06,8.638008,8.638016,99.898981,0.000007,99.898972,99.898987,101


In [12]:
df_rolling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3185 entries, 0 to 3184
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  3185 non-null   datetime64[ns]
 1   soil_mean  3185 non-null   float64       
 2   soil_std   3185 non-null   float64       
 3   soil_min   3185 non-null   float64       
 4   soil_max   3185 non-null   float64       
 5   rain_mean  3185 non-null   float64       
 6   rain_std   3185 non-null   float64       
 7   rain_min   3185 non-null   float64       
 8   rain_max   3185 non-null   float64       
 9   temp_mean  3185 non-null   float64       
 10  temp_std   3185 non-null   float64       
 11  temp_min   3185 non-null   float64       
 12  temp_max   3185 non-null   float64       
 13  humi_mean  3185 non-null   float64       
 14  humi_std   3185 non-null   float64       
 15  humi_min   3185 non-null   float64       
 16  humi_max   3185 non-null   float64       


In [13]:
nan_counts = df_rolling.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [14]:

df_rolling.to_csv(f"{save_path}", index=False)

print("✅ บันทึก เรียบร้อยแล้ว", save_path)

✅ บันทึก เรียบร้อยแล้ว ../../data/dev101_resample10T.csv
